In [88]:
import findspark 
findspark.init() 
import pyspark # only run after findspark.init() 
from pyspark.sql import SparkSession 
import pyspark.sql.functions as F
spark = SparkSession.builder.getOrCreate() 
df = spark.sql('''select 'spark' as hello ''') 
df.show()

+-----+
|hello|
+-----+
|spark|
+-----+



In [89]:
def make_a_dataset(name):
    df = spark.read.option("header", "true").option("delimiter", ";").option("inferSchema", "false").csv(name)
    return df

In [90]:
import os

In [91]:
df_weather = spark.read.option("header", "true").option("delimiter", ";").option("inferSchema", "false").csv('wetter.txt')
df_weather.printSchema()

root
 |-- STATIONS_ID: string (nullable = true)
 |-- MESS_DATUM: string (nullable = true)
 |-- QN: string (nullable = true)
 |-- PP_10: string (nullable = true)
 |-- TT_10: string (nullable = true)
 |-- TM5_10: string (nullable = true)
 |-- RF_10: string (nullable = true)
 |-- TD_10: string (nullable = true)



In [92]:
folder_1 = r'C:\Users\jk162nd\Big Data\Project'

In [93]:
files_list = list()

for root, dirs, files in os.walk(folder_1):
    for file in files:
        files_list.append(f'{root}/{file}')

In [94]:
make_a_dataset(files_list[0])

DataFrame[<html>: string]

In [95]:
df_weather.show()

+-----------+------------+-----+-------+------+------+------+------+
|STATIONS_ID|  MESS_DATUM|   QN|  PP_10| TT_10|TM5_10| RF_10| TD_10|
+-----------+------------+-----+-------+------+------+------+------+
|          3|199304281230|    1|  987.3|  24.9|  28.4|  23.0|   2.4|
|          3|199304281240|    1|  987.2|  24.9|  28.6|  21.0|   1.2|
|          3|199304281250|    1|  987.2|  25.5|  28.7|  20.0|   0.7|
|          3|199304281300|    1|  987.0|  25.8|  28.8|  20.0|   1.0|
|          3|199304281310|    1|  986.9|  25.8|  29.6|  20.0|   0.9|
|          3|199304281320|    1|  986.7|  25.7|  29.7|  19.0|   0.2|
|          3|199304281330|    1|  986.8|  26.0|  29.8|  20.0|   1.5|
|          3|199304281340|    1|  986.8|  26.1|  29.7|  18.0|   0.2|
|          3|199304281350|    1|  986.7|  27.0|  29.7|  19.0|   1.4|
|          3|199304281400|    1|  986.6|  26.5|  29.2|  17.0|  -0.4|
|          3|199304281410|    1|  986.5|  26.8|  30.1|  19.0|   1.6|
|          3|199304281420|    1|  

In [96]:
#Bauernregel: Das Wetter am Siebenschläfertag sieben Wochen bleiben mag.
#filter all temperature values on Siebenschläfertag (27.06.)
df = df_weather.filter(df_weather.MESS_DATUM.contains('19930627')).select(df_weather.MESS_DATUM, df_weather.TT_10)
df.show(200)

+------------+------+
|  MESS_DATUM| TT_10|
+------------+------+
|199306270000|  15.0|
|199306270010|  15.0|
|199306270020|  14.9|
|199306270030|  14.8|
|199306270040|  14.8|
|199306270050|  14.7|
|199306270100|  14.7|
|199306270110|  14.7|
|199306270120|  14.7|
|199306270130|  14.8|
|199306270140|  14.8|
|199306270150|  14.8|
|199306270200|  14.8|
|199306270210|  14.9|
|199306270220|  14.9|
|199306270230|  14.9|
|199306270240|  14.8|
|199306270250|  14.8|
|199306270300|  14.9|
|199306270310|  14.9|
|199306270320|  14.9|
|199306270330|  15.0|
|199306270340|  14.9|
|199306270350|  14.9|
|199306270400|  14.9|
|199306270410|  15.0|
|199306270420|  15.0|
|199306270430|  15.0|
|199306270440|  15.1|
|199306270450|  15.1|
|199306270500|  15.1|
|199306270510|  15.1|
|199306270520|  15.1|
|199306270530|  15.1|
|199306270540|  15.1|
|199306270550|  15.2|
|199306270600|  15.2|
|199306270610|  15.2|
|199306270620|  15.2|
|199306270630|  15.3|
|199306270640|  15.3|
|199306270650|  15.3|
|199306270

In [97]:
#count average temparute on Siebenschläfertag
df_avg = df.groupBy().agg(F.avg(df.TT_10).alias('TT_avg'))
df_avg.show()

+------------------+
|            TT_avg|
+------------------+
|16.721527777777776|
+------------------+



In [98]:
#filter all temperature values 27.06. to 15.08. (Siebenschläfertag + 7 weeks)
df_weeks = df_weather.filter((df_weather["MESS_DATUM"]>='199306270000') & (df_weather["MESS_DATUM"]<='199308152350')).select(df_weather.MESS_DATUM, df_weather.TT_10)
df_weeks.show(200)


+------------+------+
|  MESS_DATUM| TT_10|
+------------+------+
|199306270000|  15.0|
|199306270010|  15.0|
|199306270020|  14.9|
|199306270030|  14.8|
|199306270040|  14.8|
|199306270050|  14.7|
|199306270100|  14.7|
|199306270110|  14.7|
|199306270120|  14.7|
|199306270130|  14.8|
|199306270140|  14.8|
|199306270150|  14.8|
|199306270200|  14.8|
|199306270210|  14.9|
|199306270220|  14.9|
|199306270230|  14.9|
|199306270240|  14.8|
|199306270250|  14.8|
|199306270300|  14.9|
|199306270310|  14.9|
|199306270320|  14.9|
|199306270330|  15.0|
|199306270340|  14.9|
|199306270350|  14.9|
|199306270400|  14.9|
|199306270410|  15.0|
|199306270420|  15.0|
|199306270430|  15.0|
|199306270440|  15.1|
|199306270450|  15.1|
|199306270500|  15.1|
|199306270510|  15.1|
|199306270520|  15.1|
|199306270530|  15.1|
|199306270540|  15.1|
|199306270550|  15.2|
|199306270600|  15.2|
|199306270610|  15.2|
|199306270620|  15.2|
|199306270630|  15.3|
|199306270640|  15.3|
|199306270650|  15.3|
|199306270

In [99]:
#Delete the columns where values are missing (-999)
df_weather = df_weather.filter((df_weather.TT_10 != -999.0))

In [101]:
#schow average temperature 7 weeks after Siebenschläfertag
df_weeks_avg = df_weeks.groupBy().agg(F.avg(df.TT_10).alias('TT_weeks_avg'))
df_weeks_avg.show()


+------------------+
|      TT_weeks_avg|
+------------------+
|16.013619922092435|
+------------------+

